# Data Engineering 

### Source: [Alberto Nieto](https://github.com/Qberto/Voter-Participation-Data-Wrangling)

### The goal is to explore, visualize, and analyze U.S. presidential election participation data 

### Data available [here](https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ)

## Step 1: Load and Clean Election Data

In [1]:
# The import statements load each module
import arcgis
import pandas as pd
import os
import arcpy  # Best practice: Load arcpy last to maintain priority for namespace

In [2]:
# Make reference to the file path for the csv, which should be in the same directory as the notebook
table_csv_path = "countypres_2000-2016.csv"

# Use Pandas to read the csv into a dataframe
data_df = pd.read_csv(table_csv_path, dtype={'year': str, 'FIPS': str})  # dtype parameter specifies that year and FIPS fields are string

In [3]:
data_df.head()

,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,1001,President,Al Gore,democrat,4942.0,17208,20190722
1,2000,Alabama,AL,Autauga,1001,President,George W. Bush,republican,11993.0,17208,20190722
2,2000,Alabama,AL,Autauga,1001,President,Ralph Nader,green,160.0,17208,20190722
3,2000,Alabama,AL,Autauga,1001,President,Other,NaN,113.0,17208,20190722
4,2000,Alabama,AL,Baldwin,1003,President,Al Gore,democrat,13997.0,56480,20190722


In [4]:
data_df.shape

(50524, 11)

### Handle missing data 

In [5]:
# Set the field to check nulls for
field_to_check = "FIPS"

# Determine how many rows are in the table
rowcount = data_df.shape[0]

# Determine how many rows have null FIPS 
null_fips_rowcount = data_df.loc[data_df[field_to_check].isnull()].shape[0]

# Calculate how much of the data this represents as a percentage
percentage_null_fips = round((null_fips_rowcount / rowcount) * 100, 2)

# Use a print statement to report this information
print("There were "+str(null_fips_rowcount)+" records with null "+str(field_to_check)+" values in the data.\nThis amounts to " +str(percentage_null_fips)+"% of the available data.")

There were 64 records with null FIPS values in the data.
This amounts to 0.13% of the available data.


In [6]:
# Use the notnull function and the loc function to create a new dataframe without null FIPS records
data_df = data_df.loc[data_df['FIPS'].notnull()]

In [7]:
data_df.shape

(50460, 11)

### Explore and handle data types

In [8]:
# FIPS (Federal Information Processing Standards) is a five-digit code

# Check how many records have a FIPS value with four characters
trunc_df = data_df.loc[data_df['FIPS'].str.len() == 4]
trunc_data_per = (trunc_df.shape[0] / data_df.shape[0])*100

# Use another print statement (using the f format key) to report this information
print(f"{round(trunc_data_per, 2)}% of data ({trunc_df.shape[0]} rows) has truncated FIPS values.")

10.36% of data (5228 rows) has truncated FIPS values.


Next, we need to fix the FIPS field since the data has leading zeroes truncated by its interpretation as a numeric field. 

We can create a simple function in python to determine if the value is four characters, and append a leading zero if that's the case. 

In [9]:
# Define a helper function to fix truncated zeros, with one parameter: the value to be processed
def fix_trunc_zeros(val):
    # Use an if statement to check if there are four characters in the string representation of the value
    if len(str(val)) == 4:
        # If this is the case, return the value with an appended "0" in the front
        return "0"+str(val)
    # Otherwise...
    else:
        # Return the value itself
        return str(val)

In [10]:
# Test helper function with truncated value
fix_trunc_zeros(7042)  # You should see an appended zero: "07042"

'07042'

In [11]:
# Run helper function on the FIPS field using the apply and lambda method 
data_df['FIPS'] = data_df['FIPS'].apply(lambda x: fix_trunc_zeros(x)) 

# Print information on the operation performed, and show the first few records to confirm it worked
print(f"{round(trunc_data_per, 2)}% of data ({trunc_df.shape[0]} rows) had truncated FIPS IDs corrected.")
data_df.head()

10.36% of data (5228 rows) had truncated FIPS IDs corrected.


,year,state,state_po,county,FIPS,office,candidate,party,candidatevotes,totalvotes,version
0,2000,Alabama,AL,Autauga,01001,President,Al Gore,democrat,4942.0,17208,20190722
1,2000,Alabama,AL,Autauga,01001,President,George W. Bush,republican,11993.0,17208,20190722
2,2000,Alabama,AL,Autauga,01001,President,Ralph Nader,green,160.0,17208,20190722
3,2000,Alabama,AL,Autauga,01001,President,Other,NaN,113.0,17208,20190722
4,2000,Alabama,AL,Baldwin,01003,President,Al Gore,democrat,13997.0,56480,20190722


### Reformat the table structure

We now need to reformat the structure of the table. Currently, each record corresponds to a candidate and their votes in a particular county. We need each record to correspond to each county, with fields showing the votes for different candidates for that election year. 

**Let's break this down step by step and fully understand it.**


In essence, we need to:

1. Set a few fields aside, "locking" them from the table pivot. (set_index)
2. Pivot the table using the remaining fields. (groupby, cumcount, unstack)
3. Give the pivoted fields designations for each party. 
4. Bring the locked fields back to our table. 

**1. Setting an index using multiple fields** 

In [12]:
# Notice how running this cell uses the specified fields as row indices, 
# which prevents them from being "rotated" in the table pivot
data_df.set_index(['FIPS', 
                   'year', 
                   'county', 
                   'state', 
                   'state_po', 
                   'office'])

candidate  ...   version
FIPS  year county   state   state_po office                      ...          
01001 2000 Autauga  Alabama AL       President          Al Gore  ...  20190722
                                     President   George W. Bush  ...  20190722
                                     President      Ralph Nader  ...  20190722
                                     President            Other  ...  20190722
01003 2000 Baldwin  Alabama AL       President          Al Gore  ...  20190722
...                                                         ...  ...       ...
56043 2016 Washakie Wyoming WY       President     Donald Trump  ...  20190722
                                     President            Other  ...  20190722
56045 2016 Weston   Wyoming WY       President  Hillary Clinton  ...  20190722
                                     President     Donald Trump  ...  20190722
                                     President            Other  ...  20190722

[50460 rows x 5 columns]

**2. The built-in groupby function** allows us to perform an operation using the unique values from a specified set of fields. This is useful because we can then count how many rows exist for a given FIPS and Year combination, which essentially lets us group data by the candidate that it pertains to. See the following example: 

In [13]:
data_df.set_index(['FIPS', 
                   'year', 
                   'county', 
                   'state', 
                   'state_po', 
                   'office', 
                   data_df.groupby(['FIPS', 'year']).cumcount()+1])

candidate  ...   version
FIPS  year county   state   state_po office                        ...          
01001 2000 Autauga  Alabama AL       President 1          Al Gore  ...  20190722
                                               2   George W. Bush  ...  20190722
                                               3      Ralph Nader  ...  20190722
                                               4            Other  ...  20190722
01003 2000 Baldwin  Alabama AL       President 1          Al Gore  ...  20190722
...                                                           ...  ...       ...
56043 2016 Washakie Wyoming WY       President 2     Donald Trump  ...  20190722
                                               3            Other  ...  20190722
56045 2016 Weston   Wyoming WY       President 1  Hillary Clinton  ...  20190722
                                               2     Donald Trump  ...  20190722
                                               3            Other  ...  20190722

[50460 rows x 5 columns]

**3. The ability to unstack a table** allows us to perform the table pivot, which essentially "rotates" the table and makes rows into columns (or columns into rows).

In [14]:
data_df.set_index(['FIPS', 
                   'year', 
                   'county', 
                   'state', 
                   'state_po', 
                   'office', 
                   data_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()

candidate  ...     version
                                                             1  ...           4
FIPS  year county  state   state_po office                      ...            
01001 2000 Autauga Alabama AL       President          Al Gore  ...  20190722.0
      2004 Autauga Alabama AL       President       John Kerry  ...         NaN
      2008 Autauga Alabama AL       President     Barack Obama  ...         NaN
      2012 Autauga Alabama AL       President     Barack Obama  ...         NaN
      2016 Autauga Alabama AL       President  Hillary Clinton  ...         NaN
...                                                        ...  ...         ...
56045 2000 Weston  Wyoming WY       President          Al Gore  ...  20190722.0
      2004 Weston  Wyoming WY       President       John Kerry  ...         NaN
      2008 Weston  Wyoming WY       President     Barack Obama  ...         NaN
      2012 Weston  Wyoming WY       President     Barack Obama  ...         NaN
      2016 Weston  Wyoming WY       President  Hillary Clinton  ...         NaN

[15769 rows x 20 columns]

**4. Finally, we put it all together and rename the output columns**, using the pandas ability to rename fields and removing extraneous fields we no longer need. This produces the table format we need.

In [15]:
# Reformat the dataframe by setting a multiindex (set_index with multiple fields) and pivoting the table (unstack)
df_out = data_df.set_index(['FIPS', 
                            'year', 
                            'county', 
                            'state', 
                            'state_po', 
                            'office', 
                            data_df.groupby(['FIPS', 'year']).cumcount()+1]).unstack()


# Use the indexes for the columns to set column names (Ex: candidate_1, candidate_2, votes_1, votes_2, etc.)
df_out.columns = df_out.columns.map('{0[0]}_{0[1]}'.format)

In [16]:
df_out

candidate_1  ...   version_4
FIPS  year county  state   state_po office                      ...            
01001 2000 Autauga Alabama AL       President          Al Gore  ...  20190722.0
      2004 Autauga Alabama AL       President       John Kerry  ...         NaN
      2008 Autauga Alabama AL       President     Barack Obama  ...         NaN
      2012 Autauga Alabama AL       President     Barack Obama  ...         NaN
      2016 Autauga Alabama AL       President  Hillary Clinton  ...         NaN
...                                                        ...  ...         ...
56045 2000 Weston  Wyoming WY       President          Al Gore  ...  20190722.0
      2004 Weston  Wyoming WY       President       John Kerry  ...         NaN
      2008 Weston  Wyoming WY       President     Barack Obama  ...         NaN
      2012 Weston  Wyoming WY       President     Barack Obama  ...         NaN
      2016 Weston  Wyoming WY       President  Hillary Clinton  ...         NaN

[15769 rows x 20 columns]

In [17]:
# Rename columns 
df_out = df_out.rename(columns={"candidate_1": "candidate_dem",
                                "candidatevotes_1": "votes_dem",
                                "candidate_2": "candidate_gop",
                                "candidatevotes_2": "votes_gop",
                                "totalvotes_1": "votes_total"
                                
                                })

# Keep only the necessary columns
df_out = df_out[["candidate_dem", "votes_dem",
                 "candidate_gop", "votes_gop",
                 "votes_total"]]



In [18]:
df_out

candidate_dem  ...  votes_total
FIPS  year county  state   state_po office                      ...             
01001 2000 Autauga Alabama AL       President          Al Gore  ...      17208.0
      2004 Autauga Alabama AL       President       John Kerry  ...      20081.0
      2008 Autauga Alabama AL       President     Barack Obama  ...      23641.0
      2012 Autauga Alabama AL       President     Barack Obama  ...      23932.0
      2016 Autauga Alabama AL       President  Hillary Clinton  ...      24973.0
...                                                        ...  ...          ...
56045 2000 Weston  Wyoming WY       President          Al Gore  ...       3060.0
      2004 Weston  Wyoming WY       President       John Kerry  ...       3392.0
      2008 Weston  Wyoming WY       President     Barack Obama  ...       3414.0
      2012 Weston  Wyoming WY       President     Barack Obama  ...       3359.0
      2016 Weston  Wyoming WY       President  Hillary Clinton  ...       3526.0

[15769 rows x 5 columns]

In [19]:
# Remove the multiindex since we no longer need these fields to be "locked" for the pivot
df_out.reset_index(inplace=True)

# Rename "state_po": "state_abbrev"

df_out = df_out.rename(columns={"state_po": "state_abbrev"})

# Print out the first few records to confirm everything worked
df_out.head()

,FIPS,year,county,state,state_abbrev,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total
0,01001,2000,Autauga,Alabama,AL,President,Al Gore,4942.0,George W. Bush,11993.0,17208.0
1,01001,2004,Autauga,Alabama,AL,President,John Kerry,4758.0,George W. Bush,15196.0,20081.0
2,01001,2008,Autauga,Alabama,AL,President,Barack Obama,6093.0,John McCain,17403.0,23641.0
3,01001,2012,Autauga,Alabama,AL,President,Barack Obama,6363.0,Mitt Romney,17379.0,23932.0
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0


### Calculate Additional Columns/Features

##### Calculate total votes for non major party

In [20]:
# Calculate votes that did not go for the Democrat or Republican party
df_out['votes_other'] = df_out['votes_total'] - (df_out['votes_dem'] + df_out['votes_gop'])
df_out.head()

,FIPS,year,county,state,state_abbrev,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other
0,01001,2000,Autauga,Alabama,AL,President,Al Gore,4942.0,George W. Bush,11993.0,17208.0,273.0
1,01001,2004,Autauga,Alabama,AL,President,John Kerry,4758.0,George W. Bush,15196.0,20081.0,127.0
2,01001,2008,Autauga,Alabama,AL,President,Barack Obama,6093.0,John McCain,17403.0,23641.0,145.0
3,01001,2012,Autauga,Alabama,AL,President,Barack Obama,6363.0,Mitt Romney,17379.0,23932.0,190.0
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0,865.0


##### Create additional attributes (voter percentages and raw differences)

In [21]:
# Calculate voter share attributes
df_out['voter_share_major_party'] = (df_out['votes_dem'] + df_out['votes_gop']) / df_out['votes_total']
df_out['voter_share_dem'] = df_out['votes_dem'] / df_out['votes_total']
df_out['voter_share_gop'] = df_out['votes_gop'] / df_out['votes_total']
df_out['voter_share_other'] = df_out['votes_other'] / df_out['votes_total']

# Calculate raw difference attributes
df_out['rawdiff_dem_vs_gop'] = df_out['votes_dem'] - df_out['votes_gop']
df_out['rawdiff_gop_vs_dem'] = df_out['votes_gop'] - df_out['votes_dem']
df_out['rawdiff_dem_vs_other'] = df_out['votes_dem'] - df_out['votes_other']
df_out['rawdiff_gop_vs_other'] = df_out['votes_gop'] - df_out['votes_other']
df_out['rawdiff_other_vs_dem'] = df_out['votes_other'] - df_out['votes_dem']
df_out['rawdiff_other_vs_gop'] = df_out['votes_other'] - df_out['votes_gop']

# Calculate percent difference attributes
df_out['pctdiff_dem_vs_gop'] = (df_out['votes_dem'] - df_out['votes_gop']) / df_out['votes_total']
df_out['pctdiff_gop_vs_dem'] = (df_out['votes_gop'] - df_out['votes_dem']) / df_out['votes_total']
df_out['pctdiff_dem_vs_other'] = (df_out['votes_dem'] - df_out['votes_other']) / df_out['votes_total']
df_out['pctdiff_gop_vs_other'] = (df_out['votes_gop'] - df_out['votes_other']) / df_out['votes_total']
df_out['pctdiff_other_vs_dem'] = (df_out['votes_other'] - df_out['votes_dem']) / df_out['votes_total']
df_out['pctdiff_other_vs_gop'] = (df_out['votes_other'] - df_out['votes_gop']) / df_out['votes_total']

df_out.head()

,FIPS,year,county,state,state_abbrev,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop
0,01001,2000,Autauga,Alabama,AL,President,Al Gore,4942.0,George W. Bush,11993.0,17208.0,273.0,0.984135,0.287192,0.696943,0.015865,-7051.0,7051.0,4669.0,11720.0,-4669.0,-11720.0,-0.409751,0.409751,0.271327,0.681079,-0.271327,-0.681079
1,01001,2004,Autauga,Alabama,AL,President,John Kerry,4758.0,George W. Bush,15196.0,20081.0,127.0,0.993676,0.236940,0.756735,0.006324,-10438.0,10438.0,4631.0,15069.0,-4631.0,-15069.0,-0.519795,0.519795,0.230616,0.750411,-0.230616,-0.750411
2,01001,2008,Autauga,Alabama,AL,President,Barack Obama,6093.0,John McCain,17403.0,23641.0,145.0,0.993867,0.257730,0.736136,0.006133,-11310.0,11310.0,5948.0,17258.0,-5948.0,-17258.0,-0.478406,0.478406,0.251597,0.730003,-0.251597,-0.730003
3,01001,2012,Autauga,Alabama,AL,President,Barack Obama,6363.0,Mitt Romney,17379.0,23932.0,190.0,0.992061,0.265878,0.726183,0.007939,-11016.0,11016.0,6173.0,17189.0,-6173.0,-17189.0,-0.460304,0.460304,0.257939,0.718243,-0.257939,-0.718243
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0,865.0,0.965363,0.237697,0.727666,0.034637,-12236.0,12236.0,5071.0,17307.0,-5071.0,-17307.0,-0.489969,0.489969,0.203059,0.693028,-0.203059,-0.693028


In [22]:
df_out.shape

(15769, 28)

**Fix Shannon County Mess** for more info [here](https://www.census.gov/programs-surveys/geography/technical-documentation/county-changes.html)

Changes to Counties or County Equivalent Entities: 2010s

- Shannon County, South Dakota, was renamed as Oglala Lakota County and the county code changed from 113 to 102. 

In [23]:
df_out.loc[df_out['FIPS'] == '46113', 'FIPS'] = '46102'

## Step 2: Geoenable election data

Goals:
   - Retrieve 2016 USA county population GIS data
   - Perform join, bringing geometry to election data
   - Calculate Voter Turnout per county
   - Convert dataframe to feature class


We will now bring geometry data for each county into the table.

### Retrieve 2016 USA county population GIS data

A useful source of data is the [ArcGIS Living Atlas of the World](https://livingatlas.arcgis.com), where we can find a service containing [Voting Age Population totals (citizens aged 18+) for each county](https://www.arcgis.com/home/item.html?id=2e8aaf91178c4c91b974d0bc4234dbfa). 

In [24]:
# Authenticate with a GIS using the ArcGIS API for Python
gis = arcgis.gis.GIS()

In [25]:
from IPython.display import display

# Search for USA_Counties
items = gis.content.search("USA Counties", item_type="feature_service", outside_org=True, sort_field="numViews")
for item in items:
    display(item)

<Item title:"JHU Centers for Civic Impact Covid-19 County Cases (Daily Update)" type:Feature Layer Collection owner:CivicImpactJHU>

<Item title:"USA Counties (Generalized)" type:Feature Layer Collection owner:esri_dm>

<Item title:"USA Counties" type:Feature Layer Collection owner:esri_dm>

<Item title:"COVID-19 Cases US" type:Feature Layer Collection owner:CSSE_covid19>

<Item title:"JHU Centers for Civic Impact Covid-19 (test) - Deprecated" type:Feature Layer Collection owner:CivicImpactJHU_Test>

<Item title:"USA Current Wildfires" type:Feature Layer Collection owner:esri_livefeeds2>

<Item title:"USA Weather Watches and Warnings" type:Feature Layer Collection owner:esri_livefeeds2>

<Item title:"US Native American Lands COVID" type:Feature Layer Collection owner:CivicImpactJHU_Test>

<Item title:"Florida Health Metrics" type:Table Layer owner:Epidemiology.GIS>

<Item title:"Sonoma_County" type:Feature Layer Collection owner:sc_prmdgis>

In [26]:
# Choose the item

counties_item = items[1]
counties_item

<Item title:"USA Counties (Generalized)" type:Feature Layer Collection owner:esri_dm>

In [27]:
# Read the layer into a dataframe
counties_df = pd.DataFrame.spatial.from_layer(counties_item.layers[0])

In [28]:
counties_df.shape


(3142, 61)

In [29]:
counties_df.columns


Index(['AGE_10_14', 'AGE_15_19', 'AGE_20_24', 'AGE_25_34', 'AGE_35_44',
       'AGE_45_54', 'AGE_55_64', 'AGE_5_9', 'AGE_65_74', 'AGE_75_84',
       'AGE_85_UP', 'AGE_UNDER5', 'AMERI_ES', 'ASIAN', 'AVE_FAM_SZ',
       'AVE_HH_SZ', 'AVE_SALE12', 'AVE_SIZE12', 'BLACK', 'CNTY_FIPS',
       'CROP_ACR12', 'FAMILIES', 'FEMALES', 'FHH_CHILD', 'FID', 'FIPS',
       'GlobalID', 'HAWN_PI', 'HISPANIC', 'HOUSEHOLDS', 'HSEHLD_1_F',
       'HSEHLD_1_M', 'HSE_UNITS', 'MALES', 'MARHH_CHD', 'MARHH_NO_C',
       'MED_AGE', 'MED_AGE_F', 'MED_AGE_M', 'MHH_CHILD', 'MULT_RACE', 'NAME',
       'NO_FARMS12', 'OBJECTID', 'OTHER', 'OWNER_OCC', 'POP10_SQMI', 'POP2010',
       'POPULATION', 'POP_SQMI', 'RENTER_OCC', 'SHAPE', 'SQMI', 'STATE_FIPS',
       'STATE_NAME', 'Shape_Area', 'Shape_Leng', 'Shape__Area',
       'Shape__Length', 'VACANT', 'WHITE'],
      dtype='object')

In [30]:
counties_df = counties_df[['FIPS', 'NAME', 'STATE_FIPS', 'STATE_NAME', 'OBJECTID', 'POPULATION', 'POP_SQMI', 'SHAPE', 'Shape_Area', 'Shape_Leng', 'Shape__Area', 'Shape__Length']]
counties_df.shape

(3142, 12)

### Perform join, bringing geometry and population column to election data

We now have a dataframe with election data ('df_out') and a spatially-enabled dataframe of county voting-age population data ('counties_df'). Let's merge the datasets.

In [31]:
# Join the data to our election data table
geo_df = pd.merge(df_out, counties_df, left_on='FIPS', right_on="FIPS", how= 'inner')
# Visualize the merged data, notice the SHAPE column at the end
geo_df.shape

(15559, 39)

In [32]:
# Create a copy of the data, and perform a query
# Create a subset of the data just for the 2016 presidential election
data_2016_df = geo_df.copy()
data_2016_df.query("year == '2016'", inplace=True)
data_2016_df.head()

,FIPS,year,county,state,state_abbrev,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,NAME,STATE_FIPS,STATE_NAME,OBJECTID,POPULATION,POP_SQMI,SHAPE,Shape_Area,Shape_Leng,Shape__Area,Shape__Length
4,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0,865.0,0.965363,0.237697,0.727666,0.034637,-12236.0,12236.0,5071.0,17307.0,-5071.0,-17307.0,-0.489969,0.489969,0.203059,0.693028,-0.203059,-0.693028,Autauga,01,Alabama,1965,56319,93.2,"{'rings': [[[-9664832.76273449, 3808987.252222...",0.148903,1.884137,2.188823e+09,229109.164169
9,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,95215.0,3874.0,0.959313,0.193856,0.765457,0.040687,-54425.0,54425.0,14584.0,69009.0,-14584.0,-69009.0,-0.571601,0.571601,0.153169,0.724770,-0.153169,-0.724770,Baldwin,01,Alabama,1250,209227,127.6,"{'rings': [[[-9793119.56121114, 3654173.030111...",0.404489,3.678276,5.832085e+09,442546.872592
14,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,10469.0,144.0,0.986245,0.465278,0.520967,0.013755,-583.0,583.0,4727.0,5310.0,-4727.0,-5310.0,-0.055688,0.055688,0.451524,0.507212,-0.451524,-0.507212,Barbour,01,Alabama,1251,27072,29.9,"{'rings': [[[-9544908.60175304, 3714348.259590...",0.222431,2.218514,3.245946e+09,266874.305406
19,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,8819.0,207.0,0.976528,0.212496,0.764032,0.023472,-4864.0,4864.0,1667.0,6531.0,-1667.0,-6531.0,-0.551536,0.551536,0.189024,0.740560,-0.189024,-0.740560,Bibb,01,Alabama,1463,22932,36.6,"{'rings': [[[-9731533.81242838, 3896866.748799...",0.157736,1.852453,2.330690e+09,223825.974676
24,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,25588.0,573.0,0.977607,0.084258,0.893348,0.022393,-20703.0,20703.0,1583.0,22286.0,-1583.0,-22286.0,-0.809090,0.809090,0.061865,0.870955,-0.061865,-0.870955,Blount,01,Alabama,1966,57596,88.5,"{'rings': [[[-9681232.86521302, 4010079.855362...",0.167530,2.067456,2.503708e+09,249967.911420


In [33]:
data_2016_df.shape

(3112, 39)

### Convert dataframe to feature class

We can now finally convert our data to feature classes! The ArcGIS API for Python (which was invoked by using "import arcgis") lets us export the spatially-enabled dataframe to a feature class so we can do further analysis.

**Note: Executing the following cell may take a few minutes**

In [34]:
fgdb = r"C:\Users\dahla\Desktop\GIS utbildning\Spatial Data Science\Voter-Participation-Data-Wrangling-master\Data Engineering\Data Engineering.gdb"
out_2016_fc_name = "county_elections_pres_2016"
out_2016_fc = data_2016_df.spatial.to_featureclass(os.path.join(fgdb, out_2016_fc_name))
out_2016_fc

'C:\\Users\\dahla\\Desktop\\GIS utbildning\\Spatial Data Science\\Voter-Participation-Data-Wrangling-master\\Data Engineering\\Data Engineering.gdb\\county_elections_pres_2016'

## Step 3: Geoenrich election data


Goals:
   - Use Geoenrichment to bring demographic and socioeconomic variables to use in analysis

Geoenrichment in ArcGIS Pro allows us to add columns of data for each county that can help us analyze relationships and potentially model voter turnout. Geoenrichment can be performed using ArcPy, but we recommend that you use the GeoProcessing Enrich tool to explore potential variables that may help explain voter turnout. 

Another option is the one that we have just done. In other words, load, clean and add new data. 

**load data of The Citizen Voting Age Population (CVAP)** [here](https://www.census.gov/data/datasets/2016/dec/rdo/2012-2016-CVAP.html)

In [74]:
# CVAP_EST: The rounded estimate of the total number of United States citizens 18 years of age or older
# CVAP_MOE: The margin of error for the total number of United States citizens 18 years of age or older

# Read the data 

fp = "CVAP.csv"

data = pd.read_csv(fp, dtype={'GEOID': str, 'CVAP_EST': str, 'CVAP_MOE': str})

In [75]:
data.head()

,GEONAME,LNTITLE,GEOID,LNNUMBER,TOT_EST,TOT_MOE,ADU_EST,ADU_MOE,CIT_EST,CIT_MOE,CVAP_EST,CVAP_MOE
0,"Autauga County, Alabama",Total,05000US01001,1,55050,NaN,41195,34.0,54510,263,40690,236
1,"Autauga County, Alabama",Not Hispanic or Latino,05000US01001,2,53635,NaN,40290,35.0,53325,220,40015,184
2,"Autauga County, Alabama",American Indian or Alaska Native Alone,05000US01001,3,225,74.0,125,55.0,225,74,125,55
3,"Autauga County, Alabama",Asian Alone,05000US01001,4,485,165.0,390,106.0,340,200,245,161
4,"Autauga County, Alabama",Black or African American Alone,05000US01001,5,10115,242.0,7470,75.0,10115,242,7470,75


In [76]:
# Subset the data

data = data.loc[data['LNTITLE'] == 'Total']

In [77]:
data.head()

,GEONAME,LNTITLE,GEOID,LNNUMBER,TOT_EST,TOT_MOE,ADU_EST,ADU_MOE,CIT_EST,CIT_MOE,CVAP_EST,CVAP_MOE
0,"Autauga County, Alabama",Total,05000US01001,1,55050,NaN,41195,34.0,54510,263,40690,236
13,"Baldwin County, Alabama",Total,05000US01003,1,199510,NaN,155240,NaN,195655,761,151770,680
26,"Barbour County, Alabama",Total,05000US01005,1,26615,NaN,20880,19.0,26085,118,20375,113
39,"Bibb County, Alabama",Total,05000US01007,1,22570,NaN,17815,46.0,22345,63,17590,81
52,"Blount County, Alabama",Total,05000US01009,1,57705,NaN,44105,29.0,55925,339,42430,314


In [78]:
# Clean the data
# Drop the columns that are not useful

data = data[['GEOID', 'CVAP_EST', 'CVAP_MOE']]

In [79]:
data.head()

,GEOID,CVAP_EST,CVAP_MOE
0,05000US01001,40690,236
13,05000US01003,151770,680
26,05000US01005,20375,113
39,05000US01007,17590,81
52,05000US01009,42430,314


In [80]:
# fix the GEOID

data['GEOID'] = data['GEOID'].apply(lambda x: x[7:])

In [81]:
data.head()

,GEOID,CVAP_EST,CVAP_MOE
0,01001,40690,236
13,01003,151770,680
26,01005,20375,113
39,01007,17590,81
52,01009,42430,314


In [82]:
# change the column name

data = data.rename(columns={'GEOID':'FIPS'})

In [83]:
# merge the data

df_CVAP = pd.merge(data_2016_df, data, on= 'FIPS', how='left')

In [84]:
df_CVAP.shape

(3112, 41)

In [85]:
df_CVAP.head()

,FIPS,year,county,state,state_abbrev,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,NAME,STATE_FIPS,STATE_NAME,OBJECTID,POPULATION,POP_SQMI,SHAPE,Shape_Area,Shape_Leng,Shape__Area,Shape__Length,CVAP_EST,CVAP_MOE
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0,865.0,0.965363,0.237697,0.727666,0.034637,-12236.0,12236.0,5071.0,17307.0,-5071.0,-17307.0,-0.489969,0.489969,0.203059,0.693028,-0.203059,-0.693028,Autauga,01,Alabama,1965,56319,93.2,"{'rings': [[[-9664832.76273449, 3808987.252222...",0.148903,1.884137,2.188823e+09,229109.164169,40690,236
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,95215.0,3874.0,0.959313,0.193856,0.765457,0.040687,-54425.0,54425.0,14584.0,69009.0,-14584.0,-69009.0,-0.571601,0.571601,0.153169,0.724770,-0.153169,-0.724770,Baldwin,01,Alabama,1250,209227,127.6,"{'rings': [[[-9793119.56121114, 3654173.030111...",0.404489,3.678276,5.832085e+09,442546.872592,151770,680
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,10469.0,144.0,0.986245,0.465278,0.520967,0.013755,-583.0,583.0,4727.0,5310.0,-4727.0,-5310.0,-0.055688,0.055688,0.451524,0.507212,-0.451524,-0.507212,Barbour,01,Alabama,1251,27072,29.9,"{'rings': [[[-9544908.60175304, 3714348.259590...",0.222431,2.218514,3.245946e+09,266874.305406,20375,113
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,8819.0,207.0,0.976528,0.212496,0.764032,0.023472,-4864.0,4864.0,1667.0,6531.0,-1667.0,-6531.0,-0.551536,0.551536,0.189024,0.740560,-0.189024,-0.740560,Bibb,01,Alabama,1463,22932,36.6,"{'rings': [[[-9731533.81242838, 3896866.748799...",0.157736,1.852453,2.330690e+09,223825.974676,17590,81
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,25588.0,573.0,0.977607,0.084258,0.893348,0.022393,-20703.0,20703.0,1583.0,22286.0,-1583.0,-22286.0,-0.809090,0.809090,0.061865,0.870955,-0.061865,-0.870955,Blount,01,Alabama,1966,57596,88.5,"{'rings': [[[-9681232.86521302, 4010079.855362...",0.167530,2.067456,2.503708e+09,249967.911420,42430,314


In [86]:
# Create a new column for 'voter_turnout'

df_CVAP['voter_turnout'] = df_CVAP['votes_total'].astype(int) / df_CVAP['CVAP_EST'].astype(int)

In [87]:
df_CVAP

,FIPS,year,county,state,state_abbrev,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,NAME,STATE_FIPS,STATE_NAME,OBJECTID,POPULATION,POP_SQMI,SHAPE,Shape_Area,Shape_Leng,Shape__Area,Shape__Length,CVAP_EST,CVAP_MOE,voter_turnout
0,01001,2016,Autauga,Alabama,AL,President,Hillary Clinton,5936.0,Donald Trump,18172.0,24973.0,865.0,0.965363,0.237697,0.727666,0.034637,-12236.0,12236.0,5071.0,17307.0,-5071.0,-17307.0,-0.489969,0.489969,0.203059,0.693028,-0.203059,-0.693028,Autauga,01,Alabama,1965,56319,93.2,"{'rings': [[[-9664832.76273449, 3808987.252222...",0.148903,1.884137,2.188823e+09,229109.164169,40690,236,0.613738
1,01003,2016,Baldwin,Alabama,AL,President,Hillary Clinton,18458.0,Donald Trump,72883.0,95215.0,3874.0,0.959313,0.193856,0.765457,0.040687,-54425.0,54425.0,14584.0,69009.0,-14584.0,-69009.0,-0.571601,0.571601,0.153169,0.724770,-0.153169,-0.724770,Baldwin,01,Alabama,1250,209227,127.6,"{'rings': [[[-9793119.56121114, 3654173.030111...",0.404489,3.678276,5.832085e+09,442546.872592,151770,680,0.627364
2,01005,2016,Barbour,Alabama,AL,President,Hillary Clinton,4871.0,Donald Trump,5454.0,10469.0,144.0,0.986245,0.465278,0.520967,0.013755,-583.0,583.0,4727.0,5310.0,-4727.0,-5310.0,-0.055688,0.055688,0.451524,0.507212,-0.451524,-0.507212,Barbour,01,Alabama,1251,27072,29.9,"{'rings': [[[-9544908.60175304, 3714348.259590...",0.222431,2.218514,3.245946e+09,266874.305406,20375,113,0.513816
3,01007,2016,Bibb,Alabama,AL,President,Hillary Clinton,1874.0,Donald Trump,6738.0,8819.0,207.0,0.976528,0.212496,0.764032,0.023472,-4864.0,4864.0,1667.0,6531.0,-1667.0,-6531.0,-0.551536,0.551536,0.189024,0.740560,-0.189024,-0.740560,Bibb,01,Alabama,1463,22932,36.6,"{'rings': [[[-9731533.81242838, 3896866.748799...",0.157736,1.852453,2.330690e+09,223825.974676,17590,81,0.501364
4,01009,2016,Blount,Alabama,AL,President,Hillary Clinton,2156.0,Donald Trump,22859.0,25588.0,573.0,0.977607,0.084258,0.893348,0.022393,-20703.0,20703.0,1583.0,22286.0,-1583.0,-22286.0,-0.809090,0.809090,0.061865,0.870955,-0.061865,-0.870955,Blount,01,Alabama,1966,57596,88.5,"{'rings': [[[-9681232.86521302, 4010079.855362...",0.167530,2.067456,2.503708e+09,249967.911420,42430,314,0.603064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3107,56037,2016,Sweetwater,Wyoming,WY,President,Hillary Clinton,3231.0,Donald Trump,12154.0,17130.0,1745.0,0.898132,0.188616,0.709515,0.101868,-8923.0,8923.0,1486.0,10409.0,-1486.0,-10409.0,-0.520899,0.520899,0.086748,0.607647,-0.086748,-0.607647,Sweetwater,56,Wyoming,278,44755,4.3,"{'rings': [[[-12251087.7250076, 5098602.359683...",2.939413,7.676330,4.875622e+10,949368.698433,30565,396,0.560445
3108,56039,2016,Teton,Wyoming,WY,President,Hillary Clinton,7314.0,Donald Trump,3921.0,12627.0,1392.0,0.889760,0.579235,0.310525,0.110240,3393.0,-3393.0,5922.0,2529.0,-5922.0,-2529.0,0.268710,-0.268710,0.468995,0.200285,-0.468995,-0.200285,Teton,56,Wyoming,279,23219,5.5,"{'rings': [[[-12361750.9541678, 5355412.268422...",1.234222,4.743149,2.123677e+10,649933.862516,16335,452,0.773003
3109,56041,2016,Uinta,Wyoming,WY,President,Hillary Clinton,1202.0,Donald Trump,6154.0,8470.0,1114.0,0.868477,0.141913,0.726564,0.131523,-4952.0,4952.0,88.0,5040.0,-88.0,-5040.0,-0.584652,0.584652,0.010390,0.595041,-0.010390,-0.595041,Uinta,56,Wyoming,280,21790,10.4,"{'rings': [[[-12251087.7250076, 5098602.359683...",0.577557,3.164961,9.525333e+09,395229.959199,14355,217,0.590038
3110,56043,2016,Washakie,Wyoming,WY,President,Hillary Clinton,532.0,Donald Trump,2911.0,3814.0,371.0,0.902727,0.139486,0.763241,0.09727

In [88]:
# check the voter turnout values

df_CVAP.loc[df_CVAP['voter_turnout'] > 1]

,FIPS,year,county,state,state_abbrev,office,candidate_dem,votes_dem,candidate_gop,votes_gop,votes_total,votes_other,voter_share_major_party,voter_share_dem,voter_share_gop,voter_share_other,rawdiff_dem_vs_gop,rawdiff_gop_vs_dem,rawdiff_dem_vs_other,rawdiff_gop_vs_other,rawdiff_other_vs_dem,rawdiff_other_vs_gop,pctdiff_dem_vs_gop,pctdiff_gop_vs_dem,pctdiff_dem_vs_other,pctdiff_gop_vs_other,pctdiff_other_vs_dem,pctdiff_other_vs_gop,NAME,STATE_FIPS,STATE_NAME,OBJECTID,POPULATION,POP_SQMI,SHAPE,Shape_Area,Shape_Leng,Shape__Area,Shape__Length,CVAP_EST,CVAP_MOE,voter_turnout
271,08111,2016,San Juan,Colorado,CO,President,Hillary Clinton,265.0,Donald Trump,215.0,506.0,26.0,0.948617,0.523715,0.424901,0.051383,50.0,-50.0,239.0,189.0,-239.0,-189.0,0.098814,-0.098814,0.472332,0.373518,-0.472332,-0.373518,San Juan,08,Colorado,176,699,1.8,"{'rings': [[[-11964231.3789618, 4527207.437803...",0.106206,1.510836,1.664680e+09,185670.123521,495,78,1.022222
1776,35021,2016,Harding,New Mexico,NM,President,Hillary Clinton,156.0,Donald Trump,311.0,527.0,60.0,0.886148,0.296015,0.590133,0.113852,-155.0,155.0,96.0,251.0,-96.0,-251.0,-0.294118,0.294118,0.182163,0.476281,-0.182163,-0.476281,Harding,35,New Mexico,786,707,0.3,"{'rings': [[[-11617439.1827118, 4272321.261841...",0.546497,3.723858,8.356103e+09,454360.823288,470,91,1.121277
2643,48301,2016,Loving,Texas,TX,President,Hillary Clinton,4.0,Donald Trump,58.0,65.0,3.0,0.953846,0.061538,0.892308,0.046154,-54.0,54.0,1.0,55.0,-1.0,-55.0,-0.830769,0.830769,0.015385,0.846154,-0.015385,-0.846154,Loving,48,Texas,366,82,0.1,"{'rings': [[[-11504264.3855893, 3718440.641716...",0.162189,1.930544,2.366258e+09,228647.477628,60,26,1.083333
2648,48311,2016,McMullen,Texas,TX,President,Hillary Clinton,40.0,Donald Trump,454.0,499.0,5.0,0.989980,0.080160,0.909820,0.010020,-414.0,414.0,35.0,449.0,-35.0,-449.0,-0.829659,0.829659,0.070140,0.899800,-0.070140,-0.899800,McMullen,48,Texas,369,809,0.7,"{'rings': [[[-10947088.3480166, 3256745.973013...",0.269492,2.086627,3.794698e+09,249801.028037,460,105,1.084783


In [89]:
df_CVAP.loc[df_CVAP['voter_turnout'] > 1 , 'voter_turnout'] = 0.99

**convert our data to a feature class**

In [94]:
fgdb = r"C:\Users\dahla\Desktop\GIS utbildning\Spatial Data Science\Voter-Participation-Data-Wrangling-master\Data Engineering\Data Engineering.gdb"
out_2016_fc_name = "county_elections_pres_2016"
out_2016_fc = df_CVAP.spatial.to_featureclass(os.path.join(fgdb, out_2016_fc_name))
out_2016_fc

'C:\\Users\\dahla\\Desktop\\GIS utbildning\\Spatial Data Science\\Voter-Participation-Data-Wrangling-master\\Data Engineering\\Data Engineering.gdb\\county_elections_pres_2016'

## Step 4: Visualize and analyze

![](img/e1.png)

![](img/e2.png)

![](img/e3.png)

![](img/e4.png)

![](img/e5.png)

![](img/e6.png)

![](img/e7.png)

![](img/e8.png)

![](img/e9.png)